### From medium:

> **POST:** https://medium.com/coinmonks/multi-label-classification-blog-tags-prediction-using-nlp-b0b5ee6686fc

> **CODE:** https://github.com/neelindresh/NeelBlog/blob/master/%5BProject%20Complete%20Tag%20Prediction%20%5DtagLabelprediction.ipynb

In [88]:
import pandas as pd
import urllib3
from bs4 import BeautifulSoup
from pandas import DataFrame
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier

In [89]:
http = urllib3.PoolManager()
column = ["Title", "Body"]
dfBA = DataFrame(columns=column)
dfT = DataFrame(columns=[0, 1, 2, 3, 4])

In [90]:
def spider(link):
    blogData = http.request('GET', link)
    soup = BeautifulSoup(blogData.data, 'html.parser')
    for links in soup.find_all('div', {'class': 'postArticle-readMore'}):
        link = links.find('a').get('href')
        CrawlAndFrame(link)

In [91]:
def CrawlAndFrame(link):
    blogData = http.request('GET', link)
    soup = BeautifulSoup(blogData.data, 'html.parser')
    article = ''
    tags = []
    heading = soup.find('h1').text
    for para in soup.find_all('p'):
        p = para.text
        article = article + ' ' + p
    for mtags in soup.find_all('a', {'class': 'link u-baseColor--link'}):
        tags.append(mtags.text)
    someList = [heading, article, tuple(tags)]
    CreateDataFrame(someList)

In [92]:
def CreateDataFrame(someList):
    t = {}
    d = {'Title': [someList[0]], 'Body': [someList[1]]}
    for n in range(5):
        if len(someList[2]) > n:
            t[n] = [someList[2][n]]
        else:
            t[n] = ['0']
    toDf = DataFrame(data=d)
    global dfBA, dfT
    dfBA = dfBA.append(toDf)
    dfT = dfT.append(DataFrame(data=t))

In [93]:
dfBA.head()

,Title,Body


In [94]:
dfT.head()

,0,1,2,3,4


In [95]:
urls = [
    'https://medium.com/search?q=machine%20learning',
    'https://medium.com/search?q=deep%20learning'
]
for url in urls:
    spider(url)

/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpoo

/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/codemantra/anaconda3/lib/python3.7/site-packages/urllib3/connectionpoo

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
dfBA.head()


In [ ]:
dfT.head()

In [ ]:
df = pd.concat([dfBA, dfT], axis=1)

In [ ]:
df.head(10)

In [ ]:
stopWordList=stopwords.words('english')
stopWordList.remove('no')
stopWordList.remove('not')

In [ ]:
def removeTags(data):
    soup=BeautifulSoup(data,'html.parser')
    text=soup.get_text()
    return text

In [ ]:
import unicodedata
import spacy

In [ ]:
def removeAscendingChar(data):
    data = unicodedata.normalize('NFKD', data).encode('ascii',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return data

In [ ]:
def removeCharDigit(text):
    str = '`1234567890-=~@#$%^&*()_+[!{;":\'><.,/?"}]'
    for w in text:
        if w in str:
            text = text.replace(w, '')
    return text

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer

In [ ]:
lemma = WordNetLemmatizer()
token = ToktokTokenizer()


def lemitizeWords(text):
    words = token.tokenize(text)
    listLemma = []
    for w in words:
        x = lemma.lemmatize(w, 'v')
        listLemma.append(x)
    return text

In [ ]:
def stopWordsRemove(text):

    wordList = [x.lower().strip() for x in token.tokenize(text)]

    removedList = [x for x in wordList if not x in stopWordList]
    text = ' '.join(removedList)
    return text

In [ ]:
def PreProcessing(text):
    text = removeTags(text)
    text = removeCharDigit(text)
    text = removeAscendingChar(text)
    text = lemitizeWords(text)
    text = stopWordsRemove(text)
    return text

In [ ]:
totalText = ''
for x in df['Body']:
    ps = PreProcessing(x)
    totalText = totalText + " " + ps

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
wc = WordCloud().generate(totalText)
plt.figure(figsize=(16, 12))
plt.imshow(wc, interpolation="bilinear")

In [ ]:
import nltk
freqdist = nltk.FreqDist(token.tokenize(totalText))
freqdist
plt.figure(figsize=(16,5))
freqdist.plot(20)

In [ ]:
x = df.iloc[:, 0:2].values
y = df.iloc[:, 2:-1]

In [ ]:
okList = []
for cl in dfT.columns:
    for n in df[cl]:
        okList.append(n)
okList = list(set(okList))
del (okList[okList.index('0')])

In [ ]:
newDF = DataFrame(columns=okList)

In [ ]:
for x in range(dfT.count()[0]):
    someDict = {}
    for d in okList:
        rowdata = list(dfT.iloc[x])
        if d in rowdata:
            someDict[d] = 1
        else:
            someDict[d] = 0
    newDF = newDF.append(someDict, ignore_index=True)

In [ ]:
dfT.head()


In [ ]:
dfBA.index = range(dfBA.count()[0])
df = dfBA.join(newDF)

In [ ]:
df.head()

In [ ]:
x = df.iloc[:, 0:2].values
y = df.iloc[:, 2:-1].values
x1 = df.Title
x2 = df.Body
cv = CountVectorizer().fit(x1)
header = DataFrame(cv.transform(x1).todense(), columns=cv.get_feature_names())
cvArticle = CountVectorizer().fit(x2)
article = DataFrame(
    cvArticle.transform(x2).todense(), columns=cvArticle.get_feature_names())
x = pd.concat([header, article], axis=1)

In [ ]:
tfidfhead = TfidfTransformer().fit(header)
head = DataFrame(tfidfhead.transform(header).todense())
tfidfart = TfidfTransformer().fit(article)
art = DataFrame(tfidfart.transform(article).todense())
x = pd.concat([head, art], axis=1)

In [148]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y)
classifier = BinaryRelevance(GaussianNB())
classifier.fit(xtrain.astype(float), ytrain.astype(float))

predictions = classifier.predict(.astype(float))
predictions.toarray()

accuracy_score(ytest.astype(float), predictions)

  (0, 1)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 14)	1.0


In [136]:
new_test.values


array([[0., 0., 0., ..., 0., 0., 0.]])

In [135]:
xtrain.head(2)

,0,1,2,3,4,5,6,7,8,9,...,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930
12,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.352518,0.0,0.0,...,0.0,0.070855,0.087132,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.378431,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.011946,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
classifier = ClassifierChain(GaussianNB())
classifier.fit(xtrain.astype(float), ytrain.astype(float))
predictions = classifier.predict(xtest.astype(float))
accuracy_score(ytest.astype(float), predictions)

In [ ]:
classifier = LabelPowerset(GaussianNB())
classifier.fit(xtrain.astype(float), ytrain.astype(float))
predictions = classifier.predict(xtest.astype(float))
accuracy_score(ytest.astype(float), predictions)

In [ ]:
classifier = MLkNN(k=7)
classifier.fit(xtrain.astype(float), ytrain.astype(float))
predictions = classifier.predict(xtest.astype(float))
accuracy_score(ytest.astype(float), predictions)